In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from pathlib import Path
import geopandas as gpd

from lib import groundwater_qc

import matplotlib.pyplot as plt

%config InlineBackend.figure_formats = ['svg']

In [2]:
# load well locations
locs_gdf = gpd.read_file(
    "/Users/dillonragar/github/tnc_webapp/data/jldp_ngen_nhdhr.gpkg", layer="wells"
)

In [3]:
# load QCd well data, as dict

directory = Path(
    "/Users/dillonragar/github/TNC_dangermond/station_data/output/gw_qc_pass"
)

files = [file for file in directory.rglob("*.parquet") if "Depth" in file.name]

gw_dfs = {}

for f in files:
    df = pd.read_parquet(f)
    stn_id = df["stn_id_dendra"].iloc[0]
    gw_dfs[stn_id] = df

In [4]:
locs_gdf

,name,station_id_dendra,divide_id,id,toid,geometry
0,Cistern Water Tank,,cat-118,wb-118,nex-47,POINT (-2200628.557 1554053.509)
1,Cojo HQ Water Tank,,cat-49,wb-49,tnx-1000000012,POINT (-2203798.235 1551244.736)
2,Dangermond Alegria,64dbe91cd0708704d6d53d8c,cat-45,wb-45,nex-46,POINT (-2201366.767 1553047.530)
3,Dangermond Alexander Ramajal,64dbe9ce2148db3f6f481b1a,cat-43,wb-43,nex-44,POINT (-2199130.599 1558582.142)
4,Dangermond Buckhorn 1,64dbe9c755264579ef3ea324,cat-113,wb-113,nex-14,POINT (-2196921.555 1555734.107)
...,...,...,...,...,...,...
66,Santa Cruz Island Main Ranch South Well,60ca468be59e3ba96d7911d1,None,None,None,POINT (-2152727.446 1484582.407)
67,Santa Cruz Island Nursery Weather Station,63686470b055ac33a4843a58,None,None,None,POINT (-2148928.239 1482885.719)
68,Santa Cruz Island Pelican Pines,63645ca25d4311382ce55686,None,None,None,POINT (-2148837.160 1485931.333)
69,Santa Cruz Island Prsoner Well Gauge,63686563b055ac80f1843a5b,None,None,None,POINT (-2148837.160 1485931.333)


In [5]:
locs_gdf[locs_gdf["divide_id"] == "cat-118"]

,name,station_id_dendra,divide_id,id,toid,geometry
0,Cistern Water Tank,,cat-118,wb-118,nex-47,POINT (-2200628.557 1554053.509)
7,Dangermond Damsite Canyon,64dbe9c82148dbfa66481b12,cat-118,wb-118,nex-47,POINT (-2200647.155 1554058.948)


In [11]:
# get unique catchments
cats = locs_gdf["divide_id"].unique()


catchment_data = {}


for cat in cats:
    # get ids of wells in catchment
    print(f"catchment: {cat}")
    sub = locs_gdf[locs_gdf["divide_id"] == cat]
    # print(f"loc: {sub['name']}")

    ids_arr = sub["station_id_dendra"].values
    filt_ids = ids_arr[ids_arr != ""].tolist()
    print(f"stns in catchment: {filt_ids}")

    stn_df_list = []

    # get list of dfs for each catchment
    for id in filt_ids:
        if id in gw_dfs:  # some stations in a catchment have no data
            stn_df_list.append(gw_dfs[id])

    catchment_data[cat] = stn_df_list

catchment: cat-118
stns in catchment: ['64dbe9c82148dbfa66481b12']
catchment: cat-49
stns in catchment: []
catchment: cat-45
stns in catchment: ['64dbe91cd0708704d6d53d8c']
catchment: cat-43
stns in catchment: ['64dbe9ce2148db3f6f481b1a', '64dbe9ce5c0d5f46af694aa5']
catchment: cat-113
stns in catchment: ['64dbe9c755264579ef3ea324', '64dbe9c780818732a0c9038b']
catchment: cat-26
stns in catchment: ['64dbe9c8d07087cecfd53d8e']
catchment: cat-66
stns in catchment: ['64dbe9c880818761a2c9038d']
catchment: cat-69
stns in catchment: ['64dbe9ce02096974eea07382']
catchment: cat-31
stns in catchment: ['64dbe9c8d070876630d53d90', '64dbe9c92148db2146481b14', '64dbe9ca5c0d5f0fd8694a9f']
catchment: cat-30
stns in catchment: ['64d3cbe75c0d5f2b526942c8', '64dbe9c902096994b4a0737c', '64dbe9c9f1438d459a4dd9cb']
catchment: cat-56
stns in catchment: ['64dbe9ca02096942dfa0737e']
catchment: cat-120
stns in catchment: ['64dbe9ca5c0d5f04d5694aa1']
catchment: cat-65
stns in catchment: ['64dbe9ca552645785b3ea326

In [12]:
catchment_combined = {}

for cat in cats:
    if catchment_data[cat]:
        new_dfs = []
        print(catchment_data[cat])
        for df in catchment_data[cat]:
            df.index = df.index.round(freq="5min")
            new_dfs.append(df)

        df_out = pd.concat(new_dfs)

        catchment_combined[cat] = df_out

[                                timestamp_utc         va  \
timestamp_local                                            
2020-11-04 22:45:00 2020-11-05 06:45:00+00:00  [47.5639]   
2020-11-04 23:00:00 2020-11-05 07:00:00+00:00  [47.5631]   
2020-11-04 23:15:00 2020-11-05 07:15:00+00:00   [47.562]   
2020-11-04 23:30:00 2020-11-05 07:30:00+00:00  [47.5598]   
2020-11-04 23:45:00 2020-11-05 07:45:00+00:00  [47.5581]   
...                                       ...        ...   
2021-12-26 01:15:00 2021-12-26 09:15:00+00:00  [45.9354]   
2021-12-26 01:30:00 2021-12-26 09:30:00+00:00  [45.9374]   
2021-12-26 01:45:00 2021-12-26 09:45:00+00:00  [45.9412]   
2021-12-26 02:00:00 2021-12-26 10:00:00+00:00  [45.9433]   
2021-12-26 02:15:00 2021-12-26 10:15:00+00:00  [45.9476]   

                     DangermondEscondido2_Depth_to_Groundwater  \
timestamp_local                                                  
2020-11-04 22:45:00                                   177.8861   
2020-11-04 23:00:00 

In [13]:
catchment_combined["cat-53"]

,timestamp_utc,va,DangermondTinta1_Depth_to_Groundwater,q,stn_id_dendra,DangermondTinta2_Depth_to_Groundwater,DangermondTinta3_Depth_to_Groundwater,DangermondTinta4_Depth_to_Groundwater
timestamp_local,,,,,,,,
2009-10-28 12:45:00,2009-10-28 20:45:00+00:00,[52.644],65.4660,"{'annotation_ids': [{'_bsontype': 'ObjectID', ...",64dbe9cf808187425dc90395,NaN,NaN,NaN
2009-10-28 13:00:00,2009-10-28 21:00:00+00:00,[52.6381],65.4719,"{'annotation_ids': [{'_bsontype': 'ObjectID', ...",64dbe9cf808187425dc90395,NaN,NaN,NaN
2009-10-28 13:15:00,2009-10-28 21:15:00+00:00,[52.6359],65.4741,"{'annotation_ids': [{'_bsontype': 'ObjectID', ...",64dbe9cf808187425dc90395,NaN,NaN,NaN
2009-10-28 13:30:00,2009-10-28 21:30:00+00:00,[52.6385],65.4715,"{'annotation_ids': [{'_bsontype': 'ObjectID', ...",64dbe9cf808187425dc90395,NaN,NaN,NaN
2009-10-28 13:45:00,2009-10-28 21:45:00+00:00,[52.6334],65.4766,"{'annotation_ids': [{'_bsontype': 'ObjectID', ...",64dbe9cf808187425dc90395,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2021-09-06 22:15:00,2021-09-07 06:15:00+00:00,[71.7975],NaN,"{'annotation_ids': [{'_bsontype': 'ObjectID', ...",64dbe9d0d070877739d53d98,NaN,NaN,81.4025
2021-09-06 22:30:00,2021-09-07 06:30:00+00:00,[71.8008],NaN,"{'annotation_ids': [{'_bsontype': 'ObjectID', ...",64dbe9d0d070877739d53d98,NaN,NaN,81.3992
2021-09-06 22:45:00,2021-09-07 06:45:00+00:00,[71.8099],NaN,"{'annotation_ids': [{'_bsontype': 'ObjectID', ...",64dbe9d0d070877739d53d98,NaN,NaN,81.3901
